In [1]:
import pandas as pd
import numpy as np


In [2]:
# function to work with csv
def csv_extract(file):
    # read csv file
    df = pd.read_csv(file)
    # filter out the columns state
    df = df[df['States/UTs'] == 'Tamil Nadu']
    # remove the columns state
    df = df.drop(columns=['States/UTs'])
    # merge the columns day and month and year to form a single date column
    df['Date'] = pd.to_datetime(df[['Day', 'Month', 'Year']])
    # drop the columns day, month and year
    #df = df.drop(columns=['Day', 'Month', 'Year'])
    # convert to csv and save
    nw_path = file.split('.')[0] + '_tn.csv'
    df.to_csv(nw_path, index=False)
    return nw_path



In [114]:
# sort the csv file by date
def sort_csv(file):
    # read the csv file
    df = pd.read_csv(file)
    # sort the csv file by date
    df = df.sort_values(by='Date')
    # convert to csv and save to same file
    df.to_csv(file, index=False)
    return

In [4]:
# check for missing dates
def missing_dates(file):
    # read the csv file
    df = pd.read_csv(file)
    # ensure the 'Date' column is in datetime format
    df['Date'] = pd.to_datetime(df['Date'])
    # get the first and last date
    start_date = df['Date'].iloc[0]
    end_date = df['Date'].iloc[-1]
    # create a date range from the first to the last date
    date_range = pd.date_range(start=start_date, end=end_date)
    # get the missing dates
    missing_dates = date_range[~date_range.isin(df['Date'])]
    return missing_dates

In [5]:
# define the file paths
file_list0 = ['data/cleansing/filled/data_r_mean_filled.csv', 'data/cleansing/filled/data_r_median_filled.csv', 'data/cleansing/filled/data_r_mode_filled.csv']
# add _tn to the file name
file_list = ['data/cleansing/filled/data_r_mean_filled_tn.csv', 'data/cleansing/filled/data_r_median_filled_tn.csv', 'data/cleansing/filled/data_r_mode_filled_tn.csv']
file_list2 = ['data/cleansing/filled/data_r_mean_filled_tn_filled.csv', 'data/cleansing/filled/data_r_median_filled_tn_filled.csv', 'data/cleansing/filled/data_r_mode_filled_tn_filled.csv']


In [116]:
# print the missing dates in each file
for file in file_list0:
    csv_extract(file)

In [ ]:
print(resul)

In [117]:
# loop through the file paths
for file in file_list:
    sort_csv(file)

In [118]:
# loop through the file paths
missing_date_list=[]
for file in file_list:
    missing_date_list.append(missing_dates(file))


In [119]:
# loop through the file paths
missing_date_list2=[]
for file in file_list2:
    missing_date_list2.append(missing_dates(file))

In [120]:
print(missing_date_list)

[DatetimeIndex(['2016-09-16', '2016-12-06', '2017-01-20', '2017-04-25',
               '2017-12-26', '2018-08-08', '2020-08-09', '2020-08-16',
               '2020-08-23', '2020-08-30', '2021-05-06', '2021-05-24',
               '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28',
               '2021-05-29', '2021-05-30', '2021-05-31', '2021-06-01',
               '2021-06-02', '2021-06-03', '2021-06-04', '2021-06-05',
               '2021-06-06'],
              dtype='datetime64[ns]', freq=None), DatetimeIndex(['2016-09-16', '2016-12-06', '2017-01-20', '2017-04-25',
               '2017-12-26', '2018-08-08', '2020-08-09', '2020-08-16',
               '2020-08-23', '2020-08-30', '2021-05-06', '2021-05-24',
               '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28',
               '2021-05-29', '2021-05-30', '2021-05-31', '2021-06-01',
               '2021-06-02', '2021-06-03', '2021-06-04', '2021-06-05',
               '2021-06-06'],
              dtype='datetime64[ns]',

In [121]:
print(missing_date_list2)

[DatetimeIndex([], dtype='datetime64[ns]', freq='D'), DatetimeIndex([], dtype='datetime64[ns]', freq='D'), DatetimeIndex([], dtype='datetime64[ns]', freq='D')]


In [126]:
# code to fill the missing dates
def fill_missing(file):
    # read the csv file
    data = pd.read_csv(file)
    res_df = pd.DataFrame()
    m_dates= missing_dates(file)
    for m_date in m_dates:
        temp = data[(data["Month"] == m_date.month) &
                    (data["Year"] == m_date.year)]
        fdata = {}
        for c in temp.columns:
            if ((temp[c].dtype == 'int64' or temp[c].dtype == 'float64') and (c!='Day')):
                fdata[c] = temp[c].mean()
            elif (c=='Day'):
                fdata[c]=m_date.day
            elif (c=='Date'):
                fdata[c]=np.datetime64(m_date.__format__('%Y-%m-%d'))
            else:
                fdata[c]= temp[c].mean()
        temp_df = pd.DataFrame(fdata.values())
        # change dimension of temp_df
        temp_df = temp_df.T
        res_df=pd.concat([res_df,temp_df],ignore_index=True)
    # print the shape of 2 dataframes
    print("Original data shape: ",data.shape)
    print("Filled data shape: ",res_df.shape)
    # contvert to list of rows in 2 dataframes and concatenate and convert to dataframe
    res_df.columns = data.columns
    data = pd.concat([data,res_df],ignore_index=True)
    # convert to csv and save to new file
    nw_path = file.split('.')[0] + '_filled.csv'
    data.to_csv(nw_path, index=False)
    
    return
        
    

NameError: name 'temp' is not defined

In [127]:
# loop through the file paths
for file in file_list:
    fill_missing(file)

Original data shape:  (3146, 26)
Filled data shape:  (25, 26)
Original data shape:  (3146, 26)
Filled data shape:  (25, 26)
Original data shape:  (3146, 26)
Filled data shape:  (25, 26)


In [128]:
for file in file_list2:
    # display shape of the data
    data = pd.read_csv(file)
    print(data.shape)

(3171, 26)
(3171, 26)
(3171, 26)


In [129]:
# loop through the file paths
for file in file_list2:
    sort_csv(file)

In [131]:
# rename the files in file_list2
import os
for i in file_list2:
    # if file name contain the name mean then rename to mean_filled.csv
    if 'mean' in i:
        os.rename(i, 'data/cleansing/filled/mean_filled.csv')
    # if file name contain the name median then rename to median_filled.csv
    elif 'median' in i:
        os.rename(i, 'data/cleansing/filled/median_filled.csv')
    # if file name contain the name mode then rename to mode_filled.csv
    elif 'mode' in i:
        os.rename(i, 'data/cleansing/filled/mode_filled.csv')

In [ ]:
# print cwd
import os
print(os.getcwd())
print(res_df)